In [1]:
from customer_analyzer import CustomerAnalyzer
ca=CustomerAnalyzer(f"mssql+pyodbc://localhost/AdventureWorks2022?driver=ODBC+Driver+18+for+SQL+Server&trusted_connection=yes&TrustServerCertificate=yes")

  0%|          | 0/146 [00:00<?, ?it/s]

  0%|          | 0/146 [00:00<?, ?it/s]

KeyError: 'AvgAvgAvgSalesReasonIDIsAppropriateCILowerBound'

# Customer Analysis

### Fictitious scenario

The higher-ups at Adventureworks have a firm beleif in nurturing existing customer relations and in creating a steady flow of new customers. This philosophy applies not only to the big store customers but to private individual customers as well. 

As a consequence, a decision has been made to target 500 existing most valued private individual US-customers and give them a no strings attached, one-time, one-product, but any product 40 % discount. The same is to be done for 500 individuals in the general US-population. At lower levels in the organization, especially among the business analysts, this decision has sparked a discussion about whether it is at all possible to implement this decision in such a way that we can expect to actually make money on the sales with these discounted prices. 

A junior analyst decides to answer this question using a data driven approach where a market segmentation, based on available demographic data, is performed and an investigation of whether there are market segments, where Adventureworks can expect to be profitable even at these highly discounted prices.

### Executive summary

The goal of this analaysis is to find at least one segment of the US, private individual market that is profitable even at an across the board 40% discount rate. To this end, we segment this market based on available demographic data in the existing customer data base. These demographics are as follows:

1. Age
2. Marital Status (Single/Married)
3. Education Level
4. Income Level
5. Gender
6. Whether the person is a home owner
7. Whether the person is a car owner
8. Whether the peson has children in the household

We then proceed to calculate two quantities:

1. A profitability score for each segment, the unit of which is USD
2. A price and promotion sensitivity score between 0 and 1 for each segment.

The exact definitions of these measures are found in the detailed analysis. Since the explicitly stated purpose of the discount is to show customer appreciation and to attract new customers, we require the second quantity to be rather high, at least 0.75 at the 95% confidence level, so that we can be relatively confident that those targeted will have good use for the discount. 

It turns out that with these restrictions applied, there is exactly one segment, consisting of 508 customers, that has a positive profatibility score at the 95% confidence level: Above age 50, married, high education, high income men who own a home and a car, and who have children in the household. We recommend that this segment should be targeted for the discount. Below are the top five segments, ordered by their profitability score lower bound, with the discount taken into account. 

In [ ]:
ca.plot_top_five_segments()

Below are the top 5 selling products in the targeted segment and their respective profitability score lower bounds and sales volumes.

In [ ]:
ca.plot_top_five_products('01000111')

Although we refrain from doing so here, further profit optimization could be made by only targeting specific products for the discount.

### Detailed analysis

We begin by performing the segmentation. Using the Sales.Customers and Person.Person tables, we may construct the following dataframe of binary demographics data for US private individual customers:

In [ ]:
print(f'Number of rows: {ca.demogr_df.shape[0]}')
ca.demogr_df.head()

We are going to need some form of measure of responsiveness to price and promotion within each market segment. Using the tables Sales.SalesOrderHeader and Sales.Sales.OrderHeaderSalesReason, we construct a binary variable SalesReasonIDIsAppropriate which takes the value 1 if one of the reasons for the sale was "Price" or "Promotion" and 0 otherwise. We will later use this variable to define the price and promotion sensitivity measure. The updated dataframe looks like this: 

In [ ]:
print(f'Number of rows: {ca.demordsreason_df.shape[0]}')
ca.demordsreason_df.head()

We see now that the number of rows has increased, which is natural since one customer can have many orders. We will also need a measure of profitability within each segment. To construct such a measure, we need detailed data about the contents of the order such as order quantities, product ids and prices. This information can be found in the Sales.SalesOrderDetail table. The updated dataframe has the following appearance: 

In [ ]:
print(f'Number of rows: {ca.demordsreasonorddet_df.shape[0]}')
ca.demordsreasonorddet_df.head()

It is important to investigate the contents of the OrderQty and UnitPrice columns since these will affect how we calculate the profits. The minimum and maximum values of these are shown below

In [ ]:
print(f'OrderQty minimum: {ca.demordsreasonorddet_df.OrderQty.min()}')
print(f'OrderQty maximum: {ca.demordsreasonorddet_df.OrderQty.min()}')
print(f'UnitPriceDiscount minimum: {ca.demordsreasonorddet_df.UnitPriceDiscount.min()}')
print(f'UnitPriceDiscount maximum: {ca.demordsreasonorddet_df.UnitPriceDiscount.max()}')

It turns out that these columns both have constant values and so we don't need to take them into account. We need one more piece of data to be able to perform the analysis: The cost of each product. This information is available in the Production.Product table via the StandardCost column. We add this column to our dataframe. Then we define the measure AdjustedProfit as 0.6 * LineTotal - StandardCost. This measure takes into account the 40% discount. We compress the demographics column into a single column using concatenation to form a segment label and remove columns we don't need. The final dataframe (sorted on labels) containing the data we need now looks as follows:

In [ ]:
print(f'Number of rows: {ca.data_df.shape[0]}')
ca.data_df.head()

As can be seen, the number of rows has not changed, which is as it should be. We also note that there are rows with a negative profit. These are rows where the product sold is such that we actually loose money by giving a 40% reduction in price. We want to examine if there are any segments, where we can expect to make money even with the 40% reduction in price. There is one important thing to note here: The dataset has a hierachical structure. One customer can have many orders and one order has many products in it. It is not overly paranoid to suspect that there are intra level dependencies. For instance, perhaps it is more likely that you buy a pump, given that you buy a tyre under one and the same order. It may be the case that customers have repeated orders containing the same products. As a consequence, we cannot consider the AdjustedProfit and SalesReasonIDIsAppropriate columns of our dataframe to be i.i.d samples at the lowest levels in the hierarchy and so the central limit theorem may not apply for the calcualation of condidence intervals. To destroy intra level dependencies and end up with (close to) i.i.d samples of measures at the customer level, we proceed as follows:

1. Group at the order level and calculate the average adjusted profit and average SalesReasonIDIsAppropriate and produce a new dataframe at this level

2. Group the new dataframe at the customer level and calculate "Averages of averages" and produce a new dataframe at this level

3. The measures can now be calculated at the segment level as averages since the values at the customer level are (close to) i.i.d

4. Calculate lower bounds for these two measures at the 95% confidence level. Since there has been a bit of trickery involved, we calculate lower bounds using both the central limit theorem and by bootstrapping. We then conservatively choose the lowest of these bounds to be the bound under consideration.

We require there to be at least 50 customers in the segment to ensure central limit theorem applicability for the calculation of lower bounds. Also, we want our chosen customers to actually use the discount, because only then, can we be relatively certain that the "softer" intentions of the discount, such as the customer feeling appreciated, will be achieved. We therefore require the lower bound for price and promotion sensitivity to be at least 0.75. 

With these restrictions, the aggregated data is shown below:


In [ ]:
ca.aggregated_data_df

As can be seen, there are several segments for which the lower bound for the average adjusted profit is positive at the 95% confidence level. The segment with the highest lower bound is the segment with label '01000111'. This is the segment we should target. The description for this segment is as follows:

In [ ]:
ca.label_to_description('01000111')

To those who have, more shall be given, it would seem. We note that there is at least one segment with a substantially higher point estimate for the average adjusted profit than the one we target. The lower bound, however, is not higher. This indicates that within that segment, the "spread" of average profit over different products is higher than within our targeted segment. What products then, should we expect to sell in our targeted "alpha male" segment?

In [ ]:
tp=ca.get_top_products('01000111')
tp

We see that tires and tubes are the top sellers. We also note that among these top sellers there are products with a negative lower bound for the profit. These, however, are compensated for by the products with positive lower bound and we arrive at a positive estimated lower bound for the segment as a whole. This concludes the detailed analysis.

### Meta analysis (caveats and considerations)

We have found a market segment that is profitable, even at a discount rate of 40%. This market segment consists of men who have come of age and who are successful at life. They own high cost assets such as houses and cars. It does seem plausible that such a market segment can and will purchase products where we have a high enough profit margin that, even at a 40% discount rate, we are still profitable in the segment as a whole.  

As with any statistical analysis however, there are considerations and caveats to be made. Here are a few:

1. If the customers and patterns of purchase in our database is not a representative sample of the market, the conclusion made in this analysis may not hold true. We do however, have the odds on our side that this is not the case, since we are dealing with a large enough sample that the probability of producing a pathological sample of that size is low.

2. We are calculating our point- and interval estimates at the line item level. Since there are more line items than there are orders and more orders than there are customers, it cannot be ruled out that our sample is not truly independent. There may be dependencies between products within the same order and between orders for the same customer. Although the central limit theorem is robust to slight violations of the assumptions involved, this is a cause for some concern and warrants further investigation.

3. There is a possibility that the 40% discount will change the distribution of sales volumes over the product categories. Given a 40% discount, you may not go for "Tires and tubes", but rather for more expensive products such as bikes. We deem it unlikely that the distribution within the "Bike" category changes much though and since this distribution is evenly distributed over Road Bikes (-55.251087 lower bound for average profit) and Mountain bikes (94.726640 lower bound for average profit) we should be profitable, even if there is such an effect.

As noted above, point 2 warrants further investigation. 
